# Finding the sentiment of page edits

In [1]:
!pip install mwclient
import mwclient
#!pip install time
import time

site= mwclient.Site("en.wikipedia.org")
page= site.pages["Bitcoin"]

In [ ]:
revs= list(page.revisions())

# Shows us the latest edit in the special ordered list
revs[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
revs= sorted(revs,key =lambda rev:rev["timestamp"])

# Shows the first ever edit in our data
revs[0]

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Decides if the text is positive or negative for example:
# I love you = positive number
# I hate you = negative number
# Note: It's not perfect : we can get a few false positives and false negatives

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

find_sentiment("I like you") #returns a number close to -1

In [ ]:
edits = {}

for rev in revs:
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)

    edits[date]["edit_count"] += 1

    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))


In [ ]:
from statistics import mean

for key in edits:
  if len(edits[key]["sentiments"])>0:
     edits[key]["sentiment"] = mean(edits[key]["sentiments"])
      # gives the percentage of comments that had negative sentiment
     edits[key]["neg_sentiment"]= len([s for s in edits[key]["sentiments"] if s<0])/ len(edits[key]["sentiments"])
  else:
      # 0 is neutral sentiment
     edits[key]["sentiments"]= 0
     edits[key]["neg_sentiment"]= 0

  del edits[key]["sentiments"]


# Converting sentiment data into a dataframe

In [ ]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [ ]:
edits_df

In [ ]:
edits_df.index = pd.to_datetime(edits_df.index)

In [ ]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [ ]:
# On the days where there were no edits fill in 0
edits_df = edits_df.reindex(dates, fill_value=0)

In [ ]:
edits_df

In [ ]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [ ]:
rolling_edits = rolling_edits.dropna()
# delete rows with missing values

In [ ]:
rolling_edits

In [ ]:
rolling_edits.to_csv("wikipedia_edits.csv")

# Downloading BITCOIN price data

In [ ]:
import yfinance as yf
import os
import pandas as pd

In [ ]:
btc_ticker = yf.Ticker("BTC-USD")

In [ ]:
if os.path.exists("btc.csv"):
    btc = pd.read_csv("btc.csv", index_col=0)
else:
    btc = btc_ticker.history(period="max")
    btc.to_csv("btc.csv")

In [ ]:
btc

# Preparing data for machine learning

In [ ]:
wiki = pd.read_csv("wikipedia_edits.csv", index_col=0, parse_dates=True)
wiki